# Chainer for Theano Users

As we mentioned [here](https://chainer.org/general/2017/09/29/thank-you-theano.html), Theano stops the development in a few weeks. Many spects of Chainer were inspired by Theano's clean interface design, so that we would like to introduce Chainer here by comparing the difference from Theano. We believe that this article assists the Theano users to move to Chainer quickly.

In this post, we asume that the modules below have been imported.

In [1]:
import numpy as np

In [2]:
import theano
import theano.tensor as T

In [3]:
import chainer
import chainer.functions as F
import chainer.links as L

## Define a parametric function

A neural network basically has many parametric functions and activation functions which are called "layers" commonly. Let's see the difference between how to create a new parametric function in Theano and Chainer. In this example, to show the way to do the same thing with the two different libraries, we show how to define the 2D convolution function. But Chainer has `chainer.links.Convolution2D`, so that you don't need to write the code below to use 2D convolution as a building block of a network actually.

### Theano:

In [4]:
class TheanoConvolutionLayer(object):
    
    def __init__(self, input, filter_shape, image_shape):
        # Prepare initial values of the parameter W
        spatial_dim = np.prod(filter_shape[2:])
        fan_in = filter_shape[1] * spatial_dim
        fan_out = filter_shape[0] * spatial_dim
        scale = np.sqrt(3. / fan_in)
        
        # Create the parameter W
        W_init = np.random.uniform(-scale, scale, filter_shape)
        self.W = theano.shared(W_init.astype(np.float32), borrow=True)

        # Create the paramter b
        b_init = np.zeros((filter_shape[0],))
        self.b = theano.shared(b_init.astype(np.float32), borrow=True)

        # Describe the convolution operation
        conv_out = T.nnet.conv2d(
            input=input,
            filters=self.W,
            filter_shape=filter_shape,
            input_shape=image_shape)
        
        # Add a bias
        self.output = conv_out + self.b.dimshuffle('x', 0, 'x', 'x')
        
        # Store paramters
        self.params = [self.W, self.b]

How can we use this class? In Theano, it defines the computation as code using symbols, but doesn't perform actual computation at that time. Namely, it defines the computational graph before run. To use the defined computational graph, we need to define another operator using `theano.function` which takes input variables and output variable.

In [12]:
batchsize = 32
input_shape = (batchsize, 1, 28, 28)
filter_shape = (6, 1, 5, 5)

# Create a tensor that represents a minibatch
x = T.fmatrix('x')
input = x.reshape(input_shape)

conv = TheanoConvolutionLayer(input, filter_shape, input_shape)
f = theano.function([input], conv.output)

`conv` is the definition of how to compute the output from the first argument `input`, and `f` is the actual operator. You can pass values to `f` to compute the result of convolution like this:

In [13]:
x_data = np.random.rand(32, 1, 28, 28).astype(np.float32)

y = f(x_data)

print(y.shape, type(y))

(32, 6, 24, 24) <class 'numpy.ndarray'>


### Chainer:

What about the case in Chainer? Theano is a more general framework for scientific calculation, while Chainer focuses on neural networks. So, Chainer has many high-level APIs that enable users to write the building blocks of neural networks easier. Well, how to write the same convolution operator in Chainer?

In [14]:
class ChainerConvolutionLayer(chainer.Link):
    
    def __init__(self, filter_shape):
        super().__init__()
        with self.init_scope():
            # Specify the way of initialize
            W_init = chainer.initializers.LeCunUniform()
            b_init = chainer.initializers.Zero()
        
            # Create a parameter object
            self.W = chainer.Parameter(W_init, filter_shape)          
            self.b = chainer.Parameter(b_init, filter_shape[0])
            
    def __call__(self, x):
        return F.convolution_2d(x, self.W, self.b)

Actually, as we said at the top of this article, Chainer has pre-implemented `chainer.links.Convolution2D` class for convolution. So, you don't need to implement the code above by yourself, but it shows how to do the same thing written in Theano above.

You can create your own parametric function by defining a class inherited from `chainer.Link` as shown in the above. What computation will be applied to the input is described in `__call__` method.

Then, how to use this class?

In [15]:
chainer_conv = ChainerConvolutionLayer(filter_shape)

y = chainer_conv(x_data)

print(y.shape, type(y), type(y.array))

(32, 6, 24, 24) <class 'chainer.variable.Variable'> <class 'numpy.ndarray'>


Chainer provides many functions in `chainer.functions` and it takes NumPy array or `chainer.Variable` object as inputs. You can write arbitrary layer using those functions to make it differentiable. Note that a `chainer.Variable` object contains its actual data in `array` property.

**NOTE:**
You can write the same thing using `L.Convolution2D` like this:

In [18]:
conv_link = L.Convolution2D(in_channels=1, out_channels=6, ksize=(5, 5))

y = conv_link(x_data)

print(y.shape, type(y), type(y.array))

(32, 6, 24, 24) <class 'chainer.variable.Variable'> <class 'numpy.ndarray'>


## Use Theano function as a layer in Chainer

How to port parametric functions written in Theano to `Link`s in Chainer is shown in the above chapter. But there's an easier way to port **non-parametric functions** from Theano to Chainer.

Chainer provides [`TheanoFunction`](https://docs.chainer.org/en/latest/reference/generated/chainer.links.TheanoFunction.html?highlight=Theano) to wrap a Theano function as a `chainer.Link`. What you need to prepare is just the inputs and outputs of the Theano function you want to port to Chainer's `Link`. For example, a convolution function of Theano can be converted to a Chainer's `Link` as followings:

In [31]:
x = T.fmatrix().reshape((32, 1, 28, 28))
W = T.fmatrix().reshape((6, 1, 5, 5))
b = T.fvector().reshape((6,))
conv_out = T.nnet.conv2d(x, W) + b.dimshuffle('x', 0, 'x', 'x')

f = L.TheanoFunction(inputs=[x, W, b], outputs=[conv_out])

/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/chainer/utils/experimental.py:104: FutureWarning: chainer.links.TheanoFunction is experimental. The interface can change in the future.
  FutureWarning)


It converts the Theano computational graph into Chainer's computational graph! So it's differentiable with the Chainer APIs, and easy to use as a building block of a network written in Chainer. But it takes `W` and `b` as input arguments, so it should be noted that it doesn't keep those parameters inside.

Anyway, how to use this ported Theano function in a network in Chainer?

In [32]:
class MyNetworkWithTheanoConvolution(chainer.Chain):
    
    def __init__(self, theano_conv):
        super().__init__()
        self.theano_conv = theano_conv
        with self.init_scope():
            self.W = chainer.Parameter(shape=(6, 1, 5, 5))
            self.b = chainer.Parameter(shape=(6,))
            self.l1 = L.Linear(None, 100)
            self.l2 = L.Linear(100, 10)
            
    def __call__(self, x):
        h = self.theano_conv(x, self.W, self.b)
        h = F.relu(h)
        h = self.l1(h)
        h = F.relu(h)
        return self.l2(h)

In [33]:
model = MyNetworkWithTheanoConvolution(f)

In [34]:
y = model(x_data)

/home/shunta/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/chainer/utils/experimental.py:104: FutureWarning: chainer.functions.TheanoFunction is experimental. The interface can change in the future.
  FutureWarning)


In [35]:
y.shape

(32, 10)